In [1]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

import matplotlib.pyplot as plt

import random
from PIL import Image
import os
from contextlib import redirect_stdout
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

# Datasets

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

dir_path = '../data/processed/Combined/'

if os.path.exists(dir_path + '.DS_Store'):
    os.remove(dir_path + '.DS_Store')

image_paths = [os.path.join(dir_path, img) for img in os.listdir(dir_path)]

mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)

for img_path in image_paths:
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img)
    mean_sum += img_tensor.mean(dim=(1, 2))
    std_sum += img_tensor.std(dim=(1, 2))

mean = mean_sum / len(image_paths)
std = std_sum / len(image_paths)

mean = mean.tolist()
std = std.tolist()

print("Mean:", mean)
print("Standard deviation:", std)


Mean: [0.31156402826309204, 0.3115655779838562, 0.3115518391132355]
Standard deviation: [0.19341090321540833, 0.1934126615524292, 0.19340379536151886]


In [27]:
def pil_loader(path):
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

class Combined(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = pil_loader(img_name)
        label = self._get_label(img_name)
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)  # Convert label to tensor

    def _get_label(self, filename):
        label = -1  # Default label in case none of the conditions match
        if 'normal' in filename:
            label = 0
        elif 'benign' in filename:
            label = 1
        elif 'malignant' in filename:
            label = 2
        return label


transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

CombinedDS = Combined(root_dir=dir_path, transform=transform)

In [28]:
train_ratio = 0.72
test_ratio = 0.2
val_ratio = 0.08

class_counts = defaultdict(int)
for idx in range(len(CombinedDS)):
    _, label = CombinedDS[idx]
    class_counts[label] += 1

train_indices_dict = defaultdict(list)
test_indices_dict = defaultdict(list)
val_indices_dict = defaultdict(list)

indices = list(range(len(CombinedDS)))
random.shuffle(indices)

for label, count in class_counts.items():
    indices_for_class = [idx for idx in indices if CombinedDS[idx][1] == label]
    train_size_class = int(train_ratio * count)
    test_size_class = int(test_ratio * count)
    val_size_class = count - train_size_class - test_size_class

    train_indices_dict[label] = indices_for_class[:train_size_class]
    test_indices_dict[label] = indices_for_class[train_size_class:train_size_class + test_size_class]
    val_indices_dict[label] = indices_for_class[train_size_class + test_size_class:]

train_indices = [idx for indices in train_indices_dict.values() for idx in indices]
test_indices = [idx for indices in test_indices_dict.values() for idx in indices]
val_indices = [idx for indices in val_indices_dict.values() for idx in indices]

train_dataset, test_dataset, val_dataset = (
    torch.utils.data.Subset(CombinedDS, train_indices),
    torch.utils.data.Subset(CombinedDS, test_indices),
    torch.utils.data.Subset(CombinedDS, val_indices)
)

print("Train set size:", len(train_dataset))
print("Test set size:", len(test_dataset))
print("Validation set size:", len(val_dataset))

KeyboardInterrupt: 

# Loading & Modifying ResNet-101 Model

In [7]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)

with redirect_stdout(None):
    model.eval()

Using cache found in /Users/ahmedmahmoud/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/ahmedmahmoud/Desktop/Uni/Semester 8/GP24_Ahmed_EmbeddedBreastCancer/Code/breast-cancer-diagnosis/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ahmedmahmoud/Desktop/Uni/Semester 8/GP24_Ahmed_EmbeddedBreastCancer/Code/breast-cancer-diagnosis/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
num_features = model.fc.in_features
num_features

2048

In [9]:
num_classes = 3
new_fc_layers = [
    torch.nn.Linear(num_features, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(256, num_classes)
]
model.fc = torch.nn.Sequential(*new_fc_layers)

In [10]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [11]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

In [24]:
for data in train_dataset:
    print(data[1])

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

KeyboardInterrupt: 

In [26]:
model = model.to('mps')

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for inputs, labels in train_dataset:
        inputs = inputs.to('mps')
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    train_loss /= len(train_dataset)
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_dataset:
            inputs, labels = inputs.to('mps'), labels.to('mps')
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_dataset)
    val_losses.append(val_loss)
    val_accuracy = 100 * correct / total
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

ValueError: expected 4D input (got 3D input)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataset:
        inputs, labels = inputs.to('mps'), labels.to('mps')
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_dataset)
test_accuracy = 100 * correct / total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')